<a href="https://colab.research.google.com/github/yoshithavalluru/assignment-2/blob/main/hw2_ac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gymnasium==0.27.1 -q
!pip install gymnasium[box2d] -q
!pip install moviepy -q
!pip install -U kora -q

In [ ]:
import random
import numpy as np
import torch

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)


def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

def apply_discount(raw_reward, gamma=0.99):
    raw_reward.reverse()
    sum = 0
    discounted_rtg_reward = []
    for reward in raw_reward:
        sum = sum*gamma+reward
        discounted_rtg_reward.append(sum)
    raw_reward.reverse()
    discounted_rtg_reward.reverse()
    # Normalization
    discounted_rtg_reward = np.array(discounted_rtg_reward)
    discounted_rtg_reward = discounted_rtg_reward - np.mean(discounted_rtg_reward) / (np.std(discounted_rtg_reward) + np.finfo(np.float32).eps)
    return torch.tensor(discounted_rtg_reward, dtype=torch.float32, device=get_device())

    
# Util function to apply reward-return (cumulative reward) on a list of instant-reward (from eq 6)
def apply_return(raw_reward):
    # Compute r_reward (as a list) from raw_reward
    r_reward = [np.sum(raw_reward)]
    return torch.tensor(r_reward, dtype=torch.float32, device=get_device())  

def pad_arrays(arr_list):
    
    if not all(isinstance(arr, np.ndarray) for arr in arr_list):
        raise ValueError("arr_list must contain only numpy arrays")
    max_shape = np.max([arr.shape for arr in arr_list], axis=0)
    padded_arr_list = []
    for arr in arr_list:
        pad_width = [(0, max_shape[i] - arr.shape[i]) for i in range(len(max_shape))]
        padded_arr = np.pad(arr, pad_width, mode='constant')
        padded_arr_list.append(padded_arr)
    return padded_arr_list 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Colab Notebooks/videos
!ls

In [ ]:
import copy
import pickle
import random
import gymnasium as gym
import torch
from collections import deque, namedtuple
from gymnasium.utils.save_video import save_video
from torch import nn
from torch.optim import Adam
from torch.distributions import Categorical
import torch.nn.functional as F
import numpy as np



# Class for training an RL agent with Actor-Critic
class ACTrainer:
    def __init__(self, params):
        self.params = params
        self.env = gym.make(self.params['env_name'])
        self.agent = ACAgent(env=self.env, params=self.params)
        self.actor_net = ActorNet(input_size=self.env.observation_space.shape[0], output_size=self.env.action_space.n, hidden_dim=self.params['hidden_dim']).to(get_device())
        self.critic_net = CriticNet(input_size=self.env.observation_space.shape[0], output_size=1, hidden_dim=self.params['hidden_dim']).to(get_device())
        self.actor_optimizer = Adam(params=self.actor_net.parameters(), lr=self.params['actor_lr'])
        self.critic_optimizer = Adam(params=self.critic_net.parameters(), lr=self.params['critic_lr'])
        self.trajectory = None

    def run_training_loop(self):
        list_ro_reward = list()
        for ro_idx in range(self.params['n_rollout']):
            self.trajectory = self.agent.collect_trajectory(policy=self.actor_net)
            self.update_critic_net()
            self.estimate_advantage()
            self.update_actor_net()
            sum_of_rewards = 0
            reward_list = self.trajectory.get('reward')
            for trajectory_reward_list in reward_list:
                sum_of_rewards += apply_return(trajectory_reward_list)
            avg_ro_reward = (sum_of_rewards/len(reward_list)).item()
            print(f'End of rollout {ro_idx}: Average trajectory reward is {avg_ro_reward: 0.2f}')
            # Append average rollout reward into a list
            list_ro_reward.append(avg_ro_reward)
        # Save avg-rewards as pickle files
        pkl_file_name = self.params['exp_name'] + '.pkl'
        with open(pkl_file_name, 'wb') as f:
            pickle.dump(list_ro_reward, f)
        # Save a video of the trained agent playing
        self.generate_video()
        # Close environment
        self.env.close()

    def update_critic_net(self):
        for critic_iter_idx in range(self.params['n_critic_iter']):
            self.update_target_value()
            for critic_epoch_idx in range(self.params['n_critic_epoch']):
                critic_loss = self.estimate_critic_loss_function()
                critic_loss.backward()
                self.critic_optimizer.step()
                self.critic_optimizer.zero_grad()

    def update_target_value(self, gamma=0.99):
        self.trajectory['state_value'] = []
        self.trajectory['target_value'] = []
        for i in range(len(self.trajectory['obs'])):
            # Calculate state value for current time step
            state_value_estimate = self.critic_net(self.trajectory['obs'][i])
            self.trajectory['state_value'].append(state_value_estimate.detach().numpy())
            target_value = self.trajectory['reward'][i]
            if i < len(self.trajectory['obs']) - 1:
                target_value= gamma * self.critic_net(self.trajectory['obs'][i+1]).detach().numpy()
            if isinstance(target_value, list):
                target_value = np.array(target_value)    
            print("type is ",type(target_value))
            self.trajectory['target_value'].append(target_value);



    def estimate_advantage(self, gamma=0.99): 
       state_value = self.trajectory['state_value'];
       target_value = self.trajectory['target_value'];
       max_dim = max(arr.ndim for arr in state_value if isinstance(arr, np.ndarray))
       for i in range(len(target_value)):
           if isinstance(target_value[i], np.ndarray):
               while target_value[i].ndim < max_dim:
                   target_value[i] = np.expand_dims(target_value[i], axis=0)
               while target_value[i].ndim > max_dim:
                   target_value[i] = np.squeeze(target_value[i], axis=0)
       # Make sure all the arrays in target_value have the same shape along axis 1 as the arrays in state_value
       target_value = [arr[:, :state_value[i].shape[1]] if isinstance(arr, np.ndarray) else arr for i, arr in enumerate(target_value)]
                
       tensor1 = torch.tensor(np.concatenate(state_value, axis=0))
       tensor2 = torch.tensor(np.concatenate(target_value, axis=0))
       result = tensor2 - tensor1
       self.trajectory['advantage'] = result.detach().numpy()





    def update_actor_net(self):
        actor_loss = self.estimate_actor_loss_function()
        actor_loss.backward()
        self.actor_optimizer.step()
        self.actor_optimizer.zero_grad()

    def estimate_critic_loss_function(self):
        critic_loss = list()
        for t_idx in range(self.params['n_trajectory_per_rollout']):
            R_t = apply_discount(self.trajectory['reward'][t_idx])
            V_s = self.critic_net(self.trajectory['obs'][t_idx]).squeeze()
            loss = F.mse_loss(V_s, R_t)
            critic_loss.append(loss)
        critic_loss = torch.stack(critic_loss).mean()
        return critic_loss


    def estimate_actor_loss_function(self):
        actor_loss = list()
        for t_idx in range(self.params['n_trajectory_per_rollout']):
            advantage = apply_discount(self.trajectory['advantage'][t_idx])
            _, log_prob = self.actor_net(self.trajectory['obs'][t_idx])
            loss = -torch.mean(advantage * log_prob)
            actor_loss.append(loss)
        actor_loss = torch.stack(actor_loss).mean()
        return actor_loss


    def generate_video(self, max_frame=1000):
        self.env = gym.make(self.params['env_name'], render_mode='rgb_array_list')
        obs, _ = self.env.reset()
        for _ in range(max_frame):
            action_idx, log_prob = self.actor_net(torch.tensor(obs, dtype=torch.float32, device=get_device()))
            obs, reward, terminated, truncated, info = self.env.step(self.agent.action_space[action_idx.item()])
            if terminated or truncated:
                break
        save_video(frames=self.env.render(), video_folder=self.params['env_name'][:-3], fps=self.env.metadata['render_fps'], step_starting_index=0, episode_index=0)



class ActorNet(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim):
        super(ActorNet, self).__init__()
        self.ff_net = nn.Sequential(
            nn.Linear(input_size, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_size),
            nn.Softmax(dim=-1)
        )

    def forward(self, obs):
        action_probs = self.ff_net(obs)
        dist_obj = Categorical(action_probs)
        action_index = dist_obj.sample()
        log_prob = dist_obj.log_prob(action_index)
        return action_index, log_prob




class CriticNet(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim):
        super(CriticNet, self).__init__()
        self.ff_net = nn.Sequential(
            nn.Linear(input_size, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_size)
        )

    def forward(self, obs):
        state_value = self.ff_net(obs)
        return state_value



# Class for agent
class ACAgent:
    def __init__(self, env, params=None):
        self.env = env
        self.params = params
        self.action_space = [action for action in range(self.env.action_space.n)]

    def collect_trajectory(self, policy):
        obs, _ = self.env.reset(seed=self.params['rng_seed'])
        rollout_buffer = list()
        for _ in range(self.params['n_trajectory_per_rollout']):
            trajectory_buffer = {'obs': list(), 'log_prob': list(), 'reward': list()}
            while True:
                obs = torch.tensor(obs, dtype=torch.float32, device=get_device())
                # Save observation
                trajectory_buffer['obs'].append(obs)
                action_idx, log_prob = policy(obs)
                obs, reward, terminated, truncated, info = self.env.step(self.action_space[action_idx.item()])
                # Save log-prob and reward into the buffer
                trajectory_buffer['log_prob'].append(log_prob)
                trajectory_buffer['reward'].append(reward)
                # Check for termination criteria
                if terminated or truncated:
                    obs, _ = self.env.reset()
                    rollout_buffer.append(trajectory_buffer)
                    break
        rollout_buffer = self.serialize_trajectory(rollout_buffer)
        return rollout_buffer

    # Converts a list-of-dictionary into dictionary-of-list
    @staticmethod
    def serialize_trajectory(rollout_buffer):
        serialized_buffer = {'obs': list(), 'log_prob': list(), 'reward': list()}
        for trajectory_buffer in rollout_buffer:
            serialized_buffer['obs'].append(torch.stack(trajectory_buffer['obs']))
            serialized_buffer['log_prob'].append(torch.stack(trajectory_buffer['log_prob']))
            serialized_buffer['reward'].append(trajectory_buffer['reward'])
        return serialized_buffer

In [ ]:
# set simulation parameters
params = {
    'env_name': 'LunarLander-v2',
    'rng_seed': 6369,
    'n_rollout': 100,
    'n_trajectory_per_rollout': 60,
    'n_critic_iter': 1,
    'n_critic_epoch': 1,
    'hidden_dim': 128,
    'actor_lr': 3e-3,
    'critic_lr': 3e-4,
    'exp_name': 'LunarLander_v2_t0'
}

# Seed RNGs
seed_everything(params['rng_seed'])

# Train agent
trainer = ACTrainer(params)
trainer.run_training_loop()